In [1]:
library(plyr)

In [2]:
## Use the same data reading procedure as in metabolome heatmap script.

# Load integrals, peak ranges, and sample data
path <- "~/Documents/Projects/asf_interactions/data/"
integral_file <- 'integrals_20170323.txt'
integral_range_file <- 'coculture_peak_ranges.txt'
integrals = read.table(paste0(path,integral_file),header=FALSE,
                         sep = ",")
integral_ranges = read.table(paste0(path,integral_range_file),header=TRUE,
                             sep = ",")
master <- read.table(file=paste0(path,"merged_metadata.txt"),header=TRUE,sep='\t')

# set column names of integrals using integral_ranges met names
colnames(integrals) <- integral_ranges$met_name
mets = colnames(integrals)
# Set rownames using experiment_id. This should be ordered correctly.
integrals$experiment_id <- master$experiment_id
# merge so that classes and integrals are in one dataframe
all_data <- join(master,integrals,by="experiment_id")

# remove non-growing samples from the data
# 500 and 492 did not grow in experiment 3
# 502 did not grow in experiment 2 or 3
all_data = all_data[!(all_data$Run == 2 & grepl('502',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('502',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('500',all_data$species)),]
all_data = all_data[!(all_data$Run == 3 & grepl('492',all_data$species)),]
# remove experiment 4 (3-species subset including only 356,361,519)
all_data = all_data[all_data$Run != "4",]

Below, we test the DNA abundance data from qPCR for normality in all conditions, and for differential abundance between mono- and co-culture. We use the two-sided Mann-Whitney test and control the false discovery rate using the Benjamini-Hochberg (BH) procedure. For the group size for correction with BH, we use only samples containing the species being tested (e.g. only co-cultures that contain that species). 

In [3]:
# Test for differential DNA abundance
# positive t-statistic means the MONOCULTURE had HIGHER abundance than the co-culture being tested.
# (i.e. coculture had negative impact on growth)

# Test all groups for normality in DNA abundance
# For Shapiro-wilks test, the null hypothesis is that the population is normally distributed.
# Therefore, if p < threshold, the population in NOT normally distributed
all_normality_results = data.frame()
path <- "~/Documents/Projects/asf_interactions/results/"
species_list = c('356','360','361','492','500','502','519')
for (species_tested in species_list) {
  # subset the master dataframe by cultures that included species and are from experiments 1, 2, and 3
  contains_species = all_data[grepl(species_tested,all_data$species),]
  contains_species = contains_species[contains_species$Run %in% c(1,2,3),]
  contains_species = droplevels(contains_species)
    
  # get the correct qPCR probe for the species being tested
  qPCR_column = colnames(contains_species)[grepl(species_tested,colnames(contains_species))]
  unique_combos = as.character(unique(contains_species$species)[!(unique(contains_species$species) %in% c(species_tested))])
  
  # for each unique combo, perform the t test
  x = contains_species[contains_species$species == species_tested,][,qPCR_column]
  x_shapiro_result = shapiro.test(x)
  
  results = lapply(seq_along(unique_combos), function (n) {
    y = contains_species[contains_species$species == unique_combos[n],][,qPCR_column]
    result = wilcox.test(x,y)
    return(result)})
 
  y_shapiro_results = lapply(seq_along(unique_combos), function (n) {
    y = contains_species[contains_species$species == unique_combos[n],][,qPCR_column]
    result = shapiro.test(y)
    return(result)})
                             
  names(results) <- paste(matrix(species_tested, ncol = 2, byrow = TRUE)[,1], matrix(unlist(unique_combos), ncol = 1, byrow = TRUE), sep = " vs. ")
  names(y_shapiro_results) <- paste(matrix(species_tested, ncol = 2, byrow = TRUE)[,1], matrix(unlist(unique_combos), ncol = 1, byrow = TRUE), sep = " vs. ")
  
  # generate vector of p values for multiple testing correction
  p_init = numeric()
  p_vector = c(p_init)
  t_stat_init = numeric()
  t_stat_vector = c(t_stat_init)
  for (i in 1:length(names(results))) {
    p_vector = c(p_vector,results[[names(results)[i]]]$p.value)
    t_stat_vector = c(t_stat_vector,results[[names(results)[i]]]$statistic)
  }
  names(p_vector) = names(results)
  names(t_stat_vector) = names(results)
  
  # perform the correction
  final_p = p.adjust(p_vector,method="BH")
  print(t_stat_vector)
  print(final_p)
  resultframe = data.frame(final_p,t_stat_vector)
  
  # save the results
  write.table(resultframe,file=paste0(path,paste0(species_tested,"_diff_abundance.txt")),row.names=TRUE,sep='\t')
  
  ## compile the normality testing results
  # make placeholder vectors, then add the monoculture result
  p_init = numeric()
  p_vector = c(p_init)
  shapiro_stat_init = numeric()
  shapiro_stat_vector = c(t_stat_init)
  for (i in 1:length(names(y_shapiro_results))) {
    p_vector = c(p_vector,results[[names(y_shapiro_results)[i]]]$p.value)
    shapiro_stat_vector = c(shapiro_stat_vector,y_shapiro_results[[names(y_shapiro_results)[i]]]$statistic)
  }
  names(p_vector) = names(y_shapiro_results)
  names(shapiro_stat_vector) = names(y_shapiro_results)
  resultframe = data.frame(p_vector,shapiro_stat_vector)

  # add the monoculture test result
  single = data.frame(x_shapiro_result$p.value,x_shapiro_result$statistic)
  rownames(single) = species_tested
  colnames(single) = colnames(resultframe)
  resultframe = rbind(resultframe,single)
  
  # combine with the existing normality test results
  all_normality_results = rbind(all_normality_results,resultframe)
  
}
write.table(resultframe,file=paste0(path,paste0("DNA_abundance_normality_tests.txt")),row.names=TRUE,sep='\t')


356 vs. 356,360 356 vs. 356,500 356 vs. 356,502 356 vs. 356,492 356 vs. 356,361 
             45              34              21              24              81 
356 vs. 356,519 
             15 
356 vs. 356,360 356 vs. 356,500 356 vs. 356,502 356 vs. 356,492 356 vs. 356,361 
    0.775624376     0.683916084     0.418181818     0.775624376     0.000246812 
356 vs. 356,519 
    0.073303167 
360 vs. 356,360 360 vs. 360,500 360 vs. 360,492 360 vs. 360,502 360 vs. 360,361 
             81              36              44               0              81 
360 vs. 360,519 
             74 
360 vs. 356,360 360 vs. 360,500 360 vs. 360,492 360 vs. 360,502 360 vs. 360,361 
    0.000123406     0.327672328     0.059460539     0.013636364     0.000123406 
360 vs. 360,519 
    0.003702180 
361 vs. 360,361 361 vs. 361,492 361 vs. 361,500 361 vs. 356,361 361 vs. 361,519 
             22               4               0              47              14 
361 vs. 361,502 
              3 
361 vs. 360,361 361 

In [4]:
# How many conditions had normal distributions?
print(dim(all_normality_results))
print(dim(all_normality_results[all_normality_results$p_vector < 0.05,]))

[1] 49  2
[1] 23  2


The only monoculture in which non-normality appears is for ASF492, but >50% of the co-culture conditions appear to be non-normally distributed. Thus, we will not assume normality is present in the population for both groups in each comparison, thus a t-test is not appropriate.

In [5]:
# Test for normality in the metabolomics data for monocultures
species_list = c('356','360','361','492','500','502','519')
# get metabolite abundances for blanks
blanks = all_data[all_data$species == "0",]

# first, test for normality in the blanks.
contains_species = all_data[all_data$species=="0",]
contains_species = droplevels(contains_species)

# for each metabolite, perform a shapiro-wilks test of normality
met_p_init = numeric()
met_ps = (met_p_init)
met_t_init = numeric()
met_ts = c(met_t_init)
met_order_init = character()
met_order = c(met_order_init)
for (met in mets) {
  result = shapiro.test(contains_species[,met])
  met_ps = c(met_ps,result$p.value)
  met_ts = c(met_ts,result$statistic)
  met_order = c(met_order,met)
}

# save the results and print values to get the number of metabolites for which the null was rejected
blank_normality_test = data.frame(met_ps,met_ts,met_order)
write.table(blank_normality_test,file=paste0(path,paste0("blank_metabolite_normality_tests.txt")),row.names=TRUE,sep='\t')
print(dim(blank_normality_test))
print(dim(blank_normality_test[blank_normality_test$met_ps < 0.05,]))

# initialize empty variable to bind new results dataframes to
all_met_tests = NULL
for (species_tested in species_list) {
  # subset the master dataframe by cultures that were monocultures of the species
  contains_species = all_data[grepl(species_tested,all_data$species),]
  contains_species = droplevels(contains_species)
    
  # for each metabolite, perform a shapiro-wilks test of normality
  met_p_init = numeric()
  met_ps = (met_p_init)
  met_shapiro_init = numeric()
  met_shapiros = c(met_t_init)
  met_order_init = character()
  met_order = c(met_order_init)
  for (met in mets) {
    result = shapiro.test(contains_species[,met])
    met_ps = c(met_ps,result$p.value)
    met_shapiros = c(met_shapiros,result$statistic)
    met_order = c(met_order,met)
  }
  
  # make new df from test results
  collapse = data.frame(met_ps,met_shapiros,met_order,species_tested)
  all_met_tests = rbind(collapse,all_met_tests)
  
}

write.table(all_met_tests,file=paste0(path,paste0("all_metabolite_normality_tests.txt")),row.names=TRUE,sep='\t')
print(dim(all_met_tests))
print(dim(all_met_tests[all_met_tests$met_ps < 0.05,]))

[1] 86  3
[1] 3 3
[1] 602   4
[1] 512   4


In [6]:
# test for differential metabolite abundance in each culture group vs. blank
# get metabolite abundances for blanks
blanks = all_data[all_data$species == "0",]
# initialize empty variable to bind new results dataframes to
all_met_tests = NULL
for (species_tested in unique(all_data$species)) {
  # subset the master dataframe by cultures that contain the species of interest
  contains_species = all_data[grepl(species_tested,all_data$species),]
  contains_species = droplevels(contains_species)
  
  # for each metabolite, perform a t-test of monoculture vs. blank
  met_p_init = numeric()
  met_ps = (met_p_init)
  met_t_init = numeric()
  met_ts = c(met_t_init)
  met_order_init = character()
  met_order = c(met_order_init)
  diff_avg_init = numeric()
  diff_avgs = c(diff_avg_init)
  blank_avg_init = numeric()
  blank_avgs = c(blank_avg_init)
  group_avg_init = numeric()
  group_avgs = c(group_avg_init)
  for (met in mets) {
    result = wilcox.test(contains_species[,met],blanks[,met],exact=FALSE)
    met_ps = c(met_ps,result$p.value)
    met_ts = c(met_ts,result$statistic)
    diff_avgs = c(diff_avgs,mean(contains_species[,met]) - mean(blanks[,met]))
    blank_avgs = c(blank_avgs,mean(blanks[,met]))
    group_avgs = c(group_avgs,mean(contains_species[,met]))
    met_order = c(met_order,met)
  }
  
  # make new df from p,t,met, and combo
  collapse = data.frame(met_ps,met_ts,diff_avgs,blank_avgs,group_avgs,met_order,species_tested)
  all_met_tests = rbind(collapse,all_met_tests)
  
}

# FDR correct the p values
all_met_tests$met_ps = p.adjust(all_met_tests$met_ps,method="BH")
# save the result
write.table(all_met_tests,file=paste0(path,"all_vs_blank_diff_metabolites.txt"),row.names=FALSE,sep='\t')

# How many metabolites were differentially abundant?
dim(all_met_tests)
dim(all_met_tests[all_met_tests$met_ps < 0.05,])

[1] 2494    7

[1] 1791    7